In [ ]:
#module and extension import (Do not Change)
import pandas as pd
import numpy as np
import gspread
import re
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials
from datetime import date

In [ ]:
#initialize required variables (Do Not Change)
scope = ['https://spreadsheets.google.com/feeds',
            'https://www.googleapis.com/auth/drive']

#service account key (Do Not Change)
#google_account_key = 'UPVandLeads_Key.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name ('/ADDPATH.json', scope)

gc = gspread.authorize(credentials)

In [ ]:
#Data process 1 (jumlah UPV DATABASE1) change file name to 'PREFERRED.xlsx'
dat1 = pd.read_excel('/FILEPATH/PREFERRED.xlsx')
dat1_inhouse = dat1[dat1['Agent Type'] == 'inhouse'].sort_values("Project").drop(labels=['Leads'], axis=1)
dat1_jv = dat1[dat1['Agent Type'] == 'inhouse_jv'].sort_values("Project")
dat1_ds = dat1[dat1['Agent Type'] == 'digital_sales'].sort_values("Project").drop(labels=['Leads'], axis=1)
dat1_ex = dat1[dat1['Agent Type'] == 'External Agent'].sort_values("Project").drop(labels= ['Leads'], axis=1)

In [ ]:
#Data prcess 2 (Data DATABASE1 extract Email) Change file name to 'PREFERRED2.xlsx'
agent= pd.read_excel('/FILEPATH/PREFERRED2.xlsx')
active= agent[agent['Status'] != 'Nonactive']
email = active.drop(columns= ['Phone', 'Type', 'Captain', 'Office', 'Project', 'Tanggal Pembuatan', 'URL Seller Page'])
datmail_inhouse= pd.merge(dat1_inhouse, email, left_on='Referral Link', right_on='Unique Code').drop(columns='Nama_y')
datmail_jv= pd.merge(dat1_jv, email, left_on='Referral Link', right_on='Unique Code').drop(columns='Nama_y')
datmail_ds= pd.merge(dat1_ds, email, left_on='Referral Link', right_on='Unique Code').drop(columns='Nama_y')

In [ ]:
#Saving data 1 copy to each folder (Do Not Change)
today = date.today().strftime('%m-%d-%Y')
ext = '.xlsx'
comp = today + ext

datmail_inhouse.to_excel(r'/FILEPATH/Data1'+comp, index = False)
datmail_jv.to_excel(r'/FILEPATH/DATA2'+comp, index=False)
datmail_ds.to_excel(r'/FILEPATH/DATA3'+comp, index=False)

In [ ]:
#Data process 3 (cleaning SAP Leads data) change file name to 'PREFERRED3.csv'
dat2 = pd.read_csv('/FILEPATH/PREFFERED3.csv') #ganti path
dat2.columns.str.match("Unnamed")
dat22 =dat2.loc[:,~dat2.columns.str.match("Unnamed")]
dat2_2nd = dat22.drop(columns=['Source - ReportingX', 'Phone', 'Year Month (Lead Submission Date)'])
dat2_2nd['Source1tf'] = dat2_2nd['Lead Source Name'].str.findall('ECAT.+') #Regex
dat2_2nd['source2tf'] = dat2_2nd['Source'].str.findall('.+ecat.+') #Regex

In [ ]:
#filtering SPECIFIC Only (Do Not Change)
dat2_2nd['source_t/f'] = dat2_2nd['Source1tf'].astype(str) + '-' + dat2_2nd['source2tf'].astype(str)
dat3 = dat2_2nd[dat2_2nd['source_t/f'] != '[]-[]']
dat3_2nd = dat3.drop(columns=['Source1tf', 'source2tf'])
dat3_2nd.head(5)

In [ ]:
#leads count per agent
dat3_2nd['Leads'] = dat3_2nd['E-Mail-Marketing'].map(dat3_2nd['E-Mail-Marketing'].value_counts())
dat3_email = dat3_2nd [['E-Mail-Marketing', 'Employee Responsible - Marketing', 'Leads']].drop_duplicates(subset='E-Mail-Marketing', keep='first')

In [ ]:
#Creating Customer Journey
ney = dat3_2nd
journey = []

#making conditionals
ney2 = ney[['All Leads', 'Accepted Leads', 'Reachable Leads', 'Follow Up Leads', 'Hot Prospect Leads', 'Won Leads']]
ney['sum'] = ney2.count(axis=1)

for row in ney['sum']:
    if row == 1:
        journey.append('All Leads')
    elif row == 2:
        journey.append('Accepted Leads')
    elif row == 3:
        journey.append('Reachable Leads')
    elif row == 4:
        journey.append('Follow Up Leads')
    elif row == 5:
        journey.append('Hot Prospect Leads')
    elif row == 6:
        journey.append('WON Leads') 

    else:
        journey.append('NO JOURNEY')
        
ney['Journey'] = journey


In [ ]:
#creating SAP data Leads
leadsacc1 = ney.drop(columns= ['All Leads', 'Accepted Leads', 'Reachable Leads', 'Follow Up Leads', 'Hot Prospect Leads', 'Won Leads', 'Leads', 'sum'])

leadsacc1['email-lower'] = leadsacc1['E-Mail-Marketing'].str.lower()
agentcode_in = datmail_inhouse[['Unique Code', 'Email']]
agentcode_ds = datmail_ds[['Unique Code', 'Email']]
agentcode_in['email-lower'] = agentcode_in['Email'].str.lower()
agentcode_ds['email-lower'] = agentcode_ds['Email'].str.lower()

leadsacc2 = pd.merge(leadsacc1, agentcode_in, how= 'left')
leadsacc3= pd.merge(leadsacc2, agentcode_ds, how= 'left')

#project V2 agent
leadsv1 = pd.read_csv('/FILEPATH/FIXDATA1.csv')
leadsv2 = leadsv1[['Unique Code', 'Project V2']]
leadsv2['Project'] = leadsv2['Project V2']
sapmerge = pd.merge(leadsacc3, leadsv2, how= 'left')

#BSDTeam
team1 = pd.read_csv('/FILEPATH/FIXDATA2.csv')
team2 = team1[['Unique Code', 'BSD Team2']]
sapmerge2 = pd.merge(sapmerge, team2, how= 'left').drop(columns=['source_t/f', 'Project V2', 'email-lower']).rename(columns={'BSD Team2':'BSD Team'})
#sapmerge2.head(5)

#Lead Nature
Ln = pd.read_excel('/FILEPATH/DF.xlsx')
Ln1 = Ln[['Lead Nature', 'Type', 'No Hp']].drop_duplicates(subset='No Hp', keep='first')
sapmerge3 = pd.merge(sapmerge2, Ln1,left_on= 'Mobile', right_on= 'No Hp', how= 'left')

In [ ]:
#merging procedure to count Leads (Inhouse)
dat3_email['email-lower'] = dat3_email['E-Mail-Marketing'].str.lower()
datmail_inhouse['email-lower'] = datmail_inhouse['Email'].str.lower()

merge_in= pd.merge(dat3_email, datmail_inhouse, how= 'outer')
merge_indrop = merge_in.drop(columns= ['E-Mail-Marketing', 'email-lower', 'Referral Link', 'Employee Responsible - Marketing']).dropna(subset= ['Unique Code'])
merge_indrop['UPV'] = merge_indrop['UPV']
merge_indrop['Leads'] = merge_indrop['Leads'].fillna(0)
merge_indrop2= merge_indrop[['Unique Code', 'Nama_x', 'Project', 'Agent Type', 'Email', 'Status', 'UPV', 'Leads']]

#changing project to project V2
leadsnoproj = leadsv2.drop(columns=['Project'])
merge_in2 = pd.merge(merge_indrop2, leadsnoproj, how='left')
#adding team
merge_in3 = pd.merge(merge_in2, team2, how= 'left').drop(columns= ['Project']).rename(columns= {'BSD Team2':'BSD Team', 'Project V2':'Project', 'Nama_x':'Nama Agent'} )
merge_in3.to_excel(r'/FILEPATH/DFMerge1.xlsx'+comp, index= False)

In [ ]:
merge_in3.loc['Total'] = merge_in3.select_dtypes(pd.np.number).sum().fillna(0)

In [ ]:
#JV data CMS only
datmail_jv2 = datmail_jv[['Unique Code', 'Nama_x', 'Project', 'Agent Type', 'Email', 'Status', 'UPV', 'Leads']].rename(columns={'Nama_x':'Nama Agent'})

#finding leads on enquiries
enq2 = Ln[Ln['Is Duplicate'] != 'YES']
enq22 = enq2[enq2['Agent Type'] != 'inhouse']
enq222 = enq22[enq22['Agent Type'] != 'digital_sales']

jv_v2= datmail_jv2[['Unique Code', 'Project']].drop_duplicates(subset='Unique Code', keep= 'first')
enq_cms= pd.merge(enq222, jv_v2, how ='left')

#merging leads to jv only
enq2['Lead'] = enq2['Agent Email'].map(enq2['Agent Email'].value_counts())
enq3 = enq2[['Unique Code','Lead']].drop_duplicates(subset='Unique Code', keep= 'first')
jv = pd.merge(datmail_jv2, enq3, how= 'left').fillna(0).drop(columns=['Leads'])
jv.loc['Total'] = jv.select_dtypes(pd.np.number).sum()


In [ ]:
#merging procedure to count Leads (Digital Sales)
datmail_ds['email-lower'] = datmail_ds['Email'].str.lower()

merge_ds = pd.merge(dat3_email, datmail_ds, how= 'outer')
merge_dsdrop = merge_ds.drop(columns= ['E-Mail-Marketing', 'email-lower', 'Referral Link', 'Employee Responsible - Marketing']).dropna(subset= ['Unique Code'])
merge_dsdrop['UPV'] = merge_dsdrop['UPV']
merge_dsdrop['Leads'] = merge_dsdrop['Leads'].fillna(0)
merge_dsdrop2= merge_dsdrop[['Unique Code', 'Nama_x', 'Project', 'Agent Type', 'Email', 'Status', 'UPV', 'Leads']].rename(columns={'Nama_x':'Nama Agent'})
merge_dsdrop2.to_excel(r'/FILEPATH/DFMerge2.xlsx'+comp, index= False)

In [ ]:
merge_dsdrop2.loc['Total'] = merge_dsdrop2.select_dtypes(pd.np.number).sum().fillna(0)

In [ ]:
#Not assigned Leads
dat3_3rd = dat3_2nd[dat3_2nd['Employee Responsible - Marketing'] == 'Not assigned']
t= Ln[['No Hp', 'Type']]
Ns= dat3_3rd[['Lead', 'Lead ID', 'Lead Source Name', 'Account', 'Mobile', 'Status', 'Employee Responsible - Marketing', 'Sales Unit', 'Project/Kawasan', 'Reason for Status', 'All Leads', 'Journey']]
Na= pd.merge(Ns, t, left_on= 'Mobile', right_on='No Hp', how= 'left')
Na.loc['Total'] = Na.select_dtypes(pd.np.number).sum().fillna(0)

In [ ]:
#This is WorkSheet ID
Sheet_id = 'INPUT-GOOGLESHEET-ID'
#This is Sheet Name
Sheet_inhouse = 'Inhouse'
Sheet_JV = 'Inhouse-JV'
Sheet_DS = 'Digital-Sales'
Sheet_EX = 'External-Agent'
Sheet_NA = 'Not assigned'
Sheet_SAP = 'Data-SAP'
Sheet_CMS = 'Data-CMS'
Sheet_cmsfull= 'CMS'

#Upload Command
d2g.upload(merge_in3, Sheet_id, Sheet_inhouse, credentials= credentials, row_names= False)
d2g.upload(jv, Sheet_id, Sheet_JV, credentials= credentials, row_names= False)
d2g.upload(merge_dsdrop2, Sheet_id, Sheet_DS, credentials= credentials, row_names= False)
d2g.upload(Na, Sheet_id, Sheet_NA, credentials= credentials, row_names= False)

In [ ]:
d2g.upload(sapmerge3, Sheet_id, Sheet_SAP, credentials= credentials, row_names= False)
d2g.upload(enq_cms, Sheet_id, Sheet_CMS, credentials= credentials, row_names= False)
d2g.upload(Ln, Sheet_id, Sheet_cmsfull, credentials= credentials, row_names= False)